# TxGNN Subgraph Training & Evaluation

This notebook reproduces what the Python script `run_txgnn.py` does, but lets you run and display results directly here for your progress report.

Make sure your environment has **TxGNN**, **DGL 0.5.2**, and **PyTorch ≥ 2.2** installed, and that your dataset folder (e.g. `data/subgraphs/drug-disease-gene/`) is already prepared using `prune_subgraph.py` and `prepare_split.py`.

In [ ]:
# --- Imports and environment check ---
import torch
from txgnn import TxData, TxGNN, TxEval
import os, sys

print('Python version:', sys.version)
print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())

In [ ]:
# --- Config ---

# Path to one of your subgraph folders (edit as needed)
DATA_DIR = './data/subgraphs/drug-disease-gene'

# Device
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Seeds and save paths
SEED = 42
SAVE_MODEL_PATH = './finetune_subgraph.pt'
SAVE_EVAL_PATH = './eval_subgraph'

# Model dimensions (small for Colab / CPU)
N_HID, N_INP, N_OUT = 64, 64, 64
PROTO, PROTO_NUM = True, 2
ATTN = False
SIM_MEASURE = 'all_nodes_profile'
AGG_MEASURE = 'rarity'
NUM_WALKS, PATH_LEN = 50, 2

# Training schedule
PRETRAIN_EPOCHS = 1      # 0 to skip pretrain
PRETRAIN_LR = 1e-3
PRETRAIN_BATCH = 512
FINETUNE_EPOCHS = 30
FINETUNE_LR = 5e-4
TRAIN_PRINT_N = 10
VALID_EVERY_N = 10

In [ ]:
# --- Load data and prepare split ---
print('Loading data from:', DATA_DIR)
TxDataObj = TxData(data_folder_path=DATA_DIR)
TxDataObj.prepare_split(split='full_graph', seed=SEED)
print('Data ready.')

In [ ]:
# --- Initialize model ---
Tx = TxGNN(
    data=TxDataObj,
    weight_bias_track=False,
    proj_name='TxGNN',
    exp_name='subgraph_notebook',
    device=DEVICE,
)

Tx.model_initialize(
    n_hid=N_HID,
    n_inp=N_INP,
    n_out=N_OUT,
    proto=PROTO,
    proto_num=PROTO_NUM,
    attention=ATTN,
    sim_measure=SIM_MEASURE,
    agg_measure=AGG_MEASURE,
    num_walks=NUM_WALKS,
    path_length=PATH_LEN,
)
print('Model initialized on', DEVICE)

In [ ]:
# --- Optional pretraining ---
if PRETRAIN_EPOCHS > 0:
    print('Starting pretraining...')
    Tx.pretrain(
        n_epoch=PRETRAIN_EPOCHS,
        learning_rate=PRETRAIN_LR,
        batch_size=PRETRAIN_BATCH,
        train_print_per_n=TRAIN_PRINT_N,
    )
else:
    print('Skipping pretrain step.')

In [ ]:
# --- Finetune on drug–disease prediction ---
print('Starting finetune...')
Tx.finetune(
    n_epoch=FINETUNE_EPOCHS,
    learning_rate=FINETUNE_LR,
    train_print_per_n=TRAIN_PRINT_N,
    valid_per_n=VALID_EVERY_N,
    save_name=SAVE_MODEL_PATH,
)
print('Finetune complete. Model saved to', SAVE_MODEL_PATH)

In [ ]:
# --- Evaluate ---
print('Evaluating...')
TxE = TxEval(model=Tx)
results = TxE.eval_disease_centric(
    disease_idxs='test_set',
    show_plot=False,
    verbose=True,
    save_result=True,
    return_raw=False,
    save_name=SAVE_EVAL_PATH,
)
print('\nEvaluation summary:')
print(results)

---
### Notes
- You can re-run this notebook for different subgraph folders by changing `DATA_DIR` at the top.
- The printed output and metrics will remain visible in the `.ipynb` file after saving, so your professor can see the training/evaluation logs.
- For faster testing on CPU, reduce `FINETUNE_EPOCHS` or skip pretraining.